In [ ]:
import open3d as o3d
import numpy as np
import os
import cv2
import csv
from math import atan, sqrt, degrees
import matplotlib.cm as cm
import pandas as pd
import matplotlib.pyplot as plt
from pypcd import pypcd
from decimal import Decimal
from scipy.spatial.transform import Rotation as R

In [ ]:
image_directory          = r"/media/tmn/mySataSSD1/DATASETS/MCDVIRAL/MCDImage/ntu_day_01/img/"
point_cloud_directory    = r"/media/tmn/mySataSSD1/DATASETS/MCDVIRAL/MCDPointCloud/ntu_day_01/inB/"    
yolo_detects_directory   = r"/media/tmn/mySataSSD1/DATASETS/MCDVIRAL/NTU_World_Cars/yolov7/runs/detect/"
pose_inW                 = pd.read_csv("/media/tmn/mySataSSD1/DATASETS/MCDVIRAL/MCDPointCloud/ntu_day_01/pose_inW.csv")
pc_inW_deskewed_filename = "/media/tmn/mySataSSD1/DATASETS/MCDVIRAL/NTU_World_Cars/yolov7/world_2_new_point_cloud.pcd"
class_choice             = 0

In [ ]:
def find_nearest_number(number_list, target_number):
    nearest_number = None
    min_difference = float('inf')  # Initialize with a large value

    for num in number_list:
        difference = abs(num - target_number)
        if difference < min_difference:
            min_difference = difference
            nearest_number = num

    return nearest_number

def depth_to_color(depth, colormap=cv2.COLORMAP_JET):
    normalized_depth = (depth - np.min(depth)) / (np.max(depth) - np.min(depth))
    color_map = cv2.applyColorMap(np.uint8(255 * normalized_depth), colormap)
    return color_map

def map_value(x, a, b, c, d):
    return c + (x - a) * (d - c) / (b - a)

def distort(x, y, k1, k2, p1, p2):

    rsq = x**2 + y**2

    x_dist = x*(1 + k1*rsq + k2*(rsq**2)) + 2*p1*x*y + p2*(rsq + 2*(x**2))
    y_dist = y*(1 + k1*rsq + k2*(rsq**2)) + p1*(rsq + 2*(y**2)) + p2*x*y

    return [x_dist, y_dist]

In [ ]:
# Some user defined functions

 

# Quaternion multiplication
from evo.core.transformations import quaternion_multiply as quatmult
# Conversion between quat and rotm
from evo.core.transformations import quaternion_matrix as quat2rotm_
from evo.core.transformations import quaternion_from_matrix as rotm2quat
# Conversion between eul angles and rotm
from evo.core.transformations import euler_matrix as eul2rotm_
from evo.core.transformations import euler_from_matrix as rotm2eul_

 

def quat2rotm(q): return quat2rotm_(q)[0:3, 0:3]
def eul2rotm(e0, e1, e2): return eul2rotm_(e0, e1, e2, 'rzyx')[0:3, 0:3]
def rotm2eul(M): return rotm2eul_(M, 'rzyx')

 

# Extra definition for euler and quaternion
def eul2quat(e0, e1, e2): return rotm2quat(eul2rotm(e0, e1, e2))
def quat2eul(q): return rotm2eul(quat2rotm(q))

 

 

def pose2Tf(pose):
    q = pose[0, [6, 3, 4, 5]]
    t = np.reshape(pose[0, [0, 1, 2]], (3, 1))
    return Qt2Tf(q, t)

 

def Tf2Qt(T):
    q = rotm2quat(T[0:3, 0:3])
    t = T[0:3, 3:4]
    return q, t

 

def Qt2Tf(q, t):
    T = np.identity(4)
    T[0:3, 0:3] = quat2rotm(q)
    T[0:3, 3:4] = t
    return T

 

def Rt2Tf(R, t):
    T = np.identity(4)
    T[0:3, 0:3] = R
    T[0:3, 3:4] = t
    return T

 

def Tf2Rt(T):
    return (T[0:3, 0:3], T[0:3, 3:4])

 

def tfinv(T):
    R = T[0:3, 0:3].copy()
    t = T[0:3, 3:4].copy()
    R = R.transpose()
    t = np.dot(R, t)*(-1)
    return Rt2Tf(R, t)

 

def tfmult(T1, T2):

    R1 = T1[0:3, 0:3].copy()
    t1 = T1[0:3, 3:4].copy()

 

    R2 = T2[0:3, 0:3].copy()
    t2 = T2[0:3, 3:4].copy()

 

    R = np.dot(R1, R2)
    t = np.dot(R1, t2) + t1
    return Rt2Tf(R, t)

In [ ]:

world_points=[]

T_B_C = np.array([[-0.01507104171318285, 0.005988709007546782, 0.9998684908857283,   0.05666674638032021],
                  [ 0.9998528146297833, -0.008108860850700655, 0.015119373419753048, 0.01100640924886362],
                  [ 0.00819833998937974, 0.9999491895792147,  -0.005865618577067036, 0.11440342483060817],
                  [ 0.0, 0.0, 0.0, 1.0]])

intrinsics= [383.9298163754784, 382.9589851049001, 322.81111219908314, 250.9729552053214]

distortion_coeffs= [-0.04406471805202435, 0.04043629293187546, -1.2130737779239174e-06, 0.0018435172104962858]

d455b_fov=[90, 65] #in degrees

T_C_B = tfinv(T_B_C)

yolo_exp_file_number=2

pose_inW['t'] = pose_inW['t'].apply(lambda x: Decimal(x))

time_str=[]

for i in pose_inW['t']:

    time_str.append(str(Decimal(i))[:20])

pose_inW['time_str']=time_str

for point_cloud_file in os.listdir(point_cloud_directory):

    first_index = point_cloud_file.index('_')

    second_index = point_cloud_file.index('_', first_index + 1)

    third_index = point_cloud_file.index('_', second_index + 1)

    fourth_index = point_cloud_file.index('.')

    point_cloud_second = point_cloud_file[second_index+1:third_index]
    point_cloud_nanosec = point_cloud_file[third_index+1:fourth_index]

    closest_imgs=[]

    for filename in os.listdir(image_directory):

        index_1 = filename.index('_')
        index_2 = filename.index('_', index_1 + 1)
        index_3 = filename.index('_', index_2 + 1)
        index_4 = filename.index('.')

        if filename[index_2+1: index_3]==point_cloud_second:

            closest_imgs.append(int(filename[index_3 + 1: filename.index('.')]))

    closest_img=find_nearest_number(closest_imgs, int(point_cloud_nanosec))

    for pic in os.listdir(image_directory):

        index_1 = pic.index('_')
        index_2 = pic.index('_', index_1 + 1)
        index_3 = pic.index('_', index_2 + 1)
        index_4 = pic.index('.')

        if pic[index_2+1 :]== point_cloud_second + '_' + str(closest_img) + '.png':

            image_path=image_directory + pic

    !python3 detect.py --weights yolov7-e6e.pt --conf 0.25 --img-size 1280 --source {image_path} --save-txt

    yolo_exp_file_path = yolo_detects_directory + "exp" + str(yolo_exp_file_number)
    yolo_exp_file_number =  yolo_exp_file_number + 1

    yolo_preds_label_directory = os.path.join(yolo_exp_file_path, "labels")

    if len(os.listdir(yolo_preds_label_directory))>0:

        for file in os.listdir(yolo_exp_file_path):

            yolo_preds_file_path = os.path.join(yolo_exp_file_path, file)

            if file.endswith('.png'):
                
                yolo_pred_image_path = yolo_preds_file_path

        for label_file in os.listdir(yolo_preds_label_directory):

            yolo_pred_image_labels_path = os.path.join(yolo_preds_label_directory, label_file)

        point_cloud = pypcd.PointCloud.from_path(os.path.join(point_cloud_directory, point_cloud_file))
        points_xyz = np.asarray(point_cloud.pc_data[['x', 'y', 'z']])

        image = cv2.imread(image_path)

        with open(yolo_pred_image_labels_path, 'r') as file:
            content = file.read().split('\n')
            content.pop()
            
        coords_list=[]

        for i in content:
            if not (int(i[:2])==class_choice):
                content.remove(i)

        for object in content:

            point_cloud_coords=object.split(' ')
            point_cloud_coords[1]=float(point_cloud_coords[1])*image.shape[1]
            point_cloud_coords[2]=float(point_cloud_coords[2])*image.shape[0]
            point_cloud_coords[3]=float(point_cloud_coords[3])*image.shape[1]
            point_cloud_coords[4]=float(point_cloud_coords[4])*image.shape[0]
            coords_list.append(point_cloud_coords[1:])

        camera_points=[]
        pixel_coords=[]
        distorted_points=[]

        index=0

        for body in points_xyz:
            
            body = np.append(list(body), 1)

            camera_coord = np.dot(T_C_B, body)

            if (-45<degrees(atan(camera_coord[0]/sqrt(camera_coord[1]**2+camera_coord[2]**2)))<45) and (-32.5<degrees(atan(camera_coord[1]/sqrt(camera_coord[0]**2+camera_coord[2]**2)))<32.5) and camera_coord[2]>0:

                camera_points.append(camera_coord[:3])

                noramlized_points = (camera_coord[:3]/camera_coord[2])[:2]

                distorted_points = distort(noramlized_points[0], noramlized_points[1], distortion_coeffs[0], distortion_coeffs[1], distortion_coeffs[2], distortion_coeffs[3])

                pixel_coords.append([distorted_points[0]*intrinsics[0]+intrinsics[2], distorted_points[1]*intrinsics[1]+intrinsics[3], camera_coord[2], index])

            index = index+1

        camera_points = np.array(camera_points)
        pixel_coords = np.array(pixel_coords)

        z_coordinates = [i[2] for i in pixel_coords]

        # Normalize the z-coordinates to the range [0, 1]
        normalized_z = (z_coordinates - np.min(z_coordinates)) / (np.max(z_coordinates) - np.min(z_coordinates))

        # Replace the original z-coordinates with the normalized values
        pixel_coords[:, 2] = [int(i*255) for i in normalized_z]

        pc_index=[]

        if image is None:
            print("Error: Could not read the image")
        else:
            for coord in pixel_coords:
                x, y = int(coord[0]), int(coord[1])
                z = int(coord[2])
                bgr_color = cv2.cvtColor(np.array([[(z, 255, 255)]], dtype=np.uint8), cv2.COLOR_HSV2BGR)[0][0]
                bgr_color = (int(bgr_color[0]), int(bgr_color[1]), int(bgr_color[2]))

                for object in coords_list:
                    if x>(object[0]-(object[2]/2)) and x<(object[0]+(object[2]/2)) and y>(object[1]-(object[3]/2)) and y<(object[1]+(object[3]/2)):

                        cv2.circle(image, (x, y), radius=1, color=bgr_color, thickness=-1)

                        pc_index.append(coord[3])


        points = np.asarray(point_cloud.pc_data[['x', 'y', 'z', 'intensity']])

        detected_pc=[]

        for i in pc_index:

            detected_pc.append(list(points[int(i)]))

        detected_pc=np.array(detected_pc)

        csv_value = find_nearest_number([int(i[11:20]) for i in pose_inW['time_str']], int(point_cloud_nanosec))

        xb_world = pose_inW[pose_inW['time_str'] == (point_cloud_second + '.' + str(csv_value))]['x']
        yb_world = pose_inW[pose_inW['time_str'] == (point_cloud_second + '.' + str(csv_value))]['y']
        zb_world = pose_inW[pose_inW['time_str'] == (point_cloud_second + '.' + str(csv_value))]['z']
        qx_world = pose_inW[pose_inW['time_str'] == (point_cloud_second + '.' + str(csv_value))]['qx']
        qy_world = pose_inW[pose_inW['time_str'] == (point_cloud_second + '.' + str(csv_value))]['qy']
        qz_world = pose_inW[pose_inW['time_str'] == (point_cloud_second + '.' + str(csv_value))]['qz']
        qw_world = pose_inW[pose_inW['time_str'] == (point_cloud_second + '.' + str(csv_value))]['qw']
        
        quaternions = np.column_stack((qx_world, qy_world, qz_world, qw_world))
        rotations = R.from_quat(quaternions)

        world_data = pd.DataFrame(columns=['Points'])
        world_data.to_csv('world_data_cars.csv', index=False)

        for i in detected_pc:

            detected_points = np.column_stack((i[0], i[1], i[2]))

            try:

                rotated_points = rotations.apply(detected_points)

            except ValueError:

                continue

            rotated_points[0][0]=rotated_points[0][0]+xb_world
            rotated_points[0][1]=rotated_points[0][1]+yb_world
            rotated_points[0][2]=rotated_points[0][2]+zb_world

            i[0] = rotated_points[0][0]
            i[1] = rotated_points[0][1]
            i[2] = rotated_points[0][2]

            world_points.append(i)

        world_data['Points'] = world_points       
        world_data.to_csv('world_data_cars.csv', index=False)    
        
    else:

        continue


    
           




In [ ]:
wp = pd.read_csv('world_data_cars.csv')

import re
import ast

def extract_numbers_from_string(input_string):
    number_pattern = r'-?\d+(?:\.\d+)?(?:e[+-]?\d+)?'
    number_matches = re.findall(number_pattern, input_string)
    number_list = [float(match) if '.' in match or 'e' in match.lower() else int(match) for match in number_matches]
    return number_list



In [ ]:
all_points=[]

for i in wp['Points']:

    all_points.append(extract_numbers_from_string(i))


In [ ]:
pypcd.save_point_cloud_bin_compressed(

                pypcd.PointCloud.from_array(np.array(list(map(tuple, all_points)), dtype=[('x', '<f4'),

                                                                                               ('y', '<f4'),

                                                                                               ('z', '<f4'),

                                                                                               ('intensity', '<f4')])), pc_inW_deskewed_filename)
    